In [3]:
import pandas as pd
import numpy as np

In [69]:
%%time
twitter = pd.read_csv("preprocessed_data.csv")    #normal one used before
#twitter = pd.read_csv("sorted_preprocessed_data.csv")  #with lowest emotion scores

Wall time: 733 ms


In [70]:
twitter

,_score,tweet_id,text,identification,emotion,Category
0,299,0x2e7caf,my sunday lineup girlstrip power 💪🏾😎,train,fear,1
1,784,0x2eff99,therell never be a time than now to what you...,train,trust,2
2,358,0x3813a3,taluuluu riiiiiiiiiiiiiiiiight 🙄,train,anger,3
3,1021,0x362eaa,never give up on your dream me to make your ...,train,anticipation,4
4,367,0x286e23,pearlharper youre the most responsible adult i...,train,surprise,5
...,...,...,...,...,...,...
317658,887,0x27028e,i better go eat something because i havent eat...,train,anticipation,4
317659,574,0x30ffbb,russia had no bearing on electionpresident did...,train,anger,3
317660,91,0x29ccd4,artziiflower beingsalmankhan that meansnow u d...,train,sadness,8
317661,532,0x345ac6,talkmaster now in greenville schigh of enjoy...,train,fear,1


In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer

#Here we get the TF_IDF for the first 100 features:
number_features = 69000

tkzr = TweetTokenizer(strip_handles = True)

tfidf_vectorizer = TfidfVectorizer(
                            lowercase=True,
                            max_features= number_features,
                            #max_df=0.8,
                            #min_df=2,
                            #ngram_range = (1,3),
                            #stop_words = "english",
                            tokenizer = tkzr.tokenize
                            )

vectors = tfidf_vectorizer.fit_transform(twitter['text'])

feature_names = tfidf_vectorizer.get_feature_names()

In [72]:
# validate: split into train and test
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(vectors, twitter['emotion'], test_size=0.2, random_state=42)

# model
lr = LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=1000,
                   multi_class='multinomial', n_jobs=-1, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=2,
                   warm_start=False)

#the ones that first worked were -> solver: liblinear  ;  multi_class: auto

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)


print('Accuracy Score:', accuracy_score(y_test, y_pred))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


convergence after 63 epochs took 16 seconds
Accuracy Score: 0.4453118851620418


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   15.9s finished


In [73]:
twitter_test_data = pd.read_csv("twitter_test_data.csv")

In [74]:
x_test_twitter = tfidf_vectorizer.transform(twitter_test_data['text'])

In [75]:
new_predict = lr.predict(x_test_twitter)

In [77]:
upload_df = pd.DataFrame(columns = [["id","emotion"]])
upload_df["id"] = twitter_test_data["tweet_id"]
upload_df["emotion"] = new_predict
upload_df

,id,emotion
0,0x28b412,anticipation
1,0x2de201,anticipation
2,0x218443,joy
3,0x2939d5,joy
4,0x26289a,fear
...,...,...
411967,0x2913b4,anticipation
411968,0x2a980e,anticipation
411969,0x316b80,anger
411970,0x29d0cb,joy


In [61]:
upload_df.to_csv("./uploads/logistic_try_1.csv",index=False)